# HydroHomies Plots
In this notebook, the plots, figures and also some explanations or details about each of them are being presented.  

To clarify plots, please follow this order:
- Title for each plot is mandatory
- Analysis must be written 
- legends are manedatory

### Loading all data


In [28]:
import yaml
import pandas as pd
with open('config.yaml') as stream:
    config = yaml.safe_load(stream)

### Cleaning (Digit Span Raw Data)

In [29]:
def clean_digit_span(raw_df):
    # Select the sequence length data from the raw data and create a dataframe
    seq_length_df = raw_df[raw_df[1].astype(str).str.match(r'\d+')]
    seq_length_df

    # Get the value of the longest sequence remebered
    longest = seq_length_df[2]
    longest = longest.tolist()

    # Get the number of errors made
    error_number = seq_length_df[3]
    error_number = error_number.tolist()

    click_stim_df = raw_df[raw_df[1]=='clickedStim']
    click_stim_df.size

    clicks_observed = click_stim_df.count(axis=1) - 2 
    clicks_observed = clicks_observed.tolist()

    clicks_expected =  pd.to_numeric(longest) + 1
    clicks_expected = clicks_expected.tolist()

    clean_data = pd.DataFrame(data ={'seq length':longest,
                        'errors': error_number,
                        'clicks expected': clicks_expected,
                        'clicks observed':clicks_observed})
    
    return clean_data


### Data Integration For Each Test

In [30]:

def create_merged_df(config_dict):
    data_dict = {}

     # selects only the files we want to use
    files = {name: file for name, file in config_dict.items()} 

    # read the files 
    for test, file in files.items():
        df_dict = pd.read_excel(file, sheet_name=None, header=None)

        for session, df in df_dict.items():
            # extracting the participant name and type name
            participant = test.split('_')[-1]
            test_name = test.split('_')[0]

            #extracting repeat number and making its column except for personal
            try:
                type, repeat = session.split('_')
                df.insert(0, 'repeat', repeat)
            except ValueError:
                type = session

            # inserting the type and participant columns
            df.insert(0, 'type', type)
            df.insert(0, 'participant', participant)

            #cleaning digit_span row data
            if test_name == 'digit_span':
                try:
                    df = clean_digit_span(df.iloc[3:])
                except:
                    continue         

            # concatenating data frames of each test
            if test_name not in data_dict:
                data_dict[test_name] = df
            else:
                data_dict[test_name] = pd.concat([data_dict[test_name], df])
    
    return data_dict

---

### Flanker Test Analysis

In [31]:
def flanker_test(flanker_df): # Roya
    pass

### Stroop Test  Analysis

In [32]:
def stroop_test(stroop_df): # Mahdiye
    pass

### Stop Signal Analysis

In [33]:
def stop_test(stop_df): # Jacob
    pass

### Verbal Fluency Analysis

In [34]:
def verbal_test(verbal_df): # Jacob
    pass

### Digit Span Analysis

In [35]:
def digit_test(digit_df): # Karina
    pass

---